<a href="https://colab.research.google.com/github/AbirAhmed72/Decision-Table-Generator/blob/decision_table_feature/decision_table_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Library Imports**

In [130]:
import pandas as pd
import itertools
import re
import string
import pandas as pd
import spacy

# **Inputs**

In [120]:
# sentences = [
#     "If the working hours are 48 or less then give normal salary.",
#     "The rate of the salary is 1.25 times if the working hours exceed 48 on normal working days.",
#     "If the working days are holidays or Sundays then the rate of the salary is 2.00 times."
# ]
sentences = [
    "If the order is from DGS&D then the discount is 10%.",
    "The discount is 15% if the order is from agents and the amount is more than Rs 50000.",
    "The discount is 10% if the order is from retailer and the amount is more than Rs 50000.",
    "If the amount is between Rs 20000 to Rs 50000 and the order is from agents then the discount is 12%.",
    "If the amount is between Rs 20000 to Rs 50000 and the order is from retailer then the discount is 8%.",
    "If the amount is less than Rs 20000 and the order is from agents then the discount is 8%.",
    "The discount is 5% if the order is from retailer and the amount is less than Rs 20000.",
    "The discount is 10% if item is furniture.",
]

# **Condition and action stubs generation**

In [121]:
def create_single_conditions(mixed_conditions):
  mixed_conditions=[item.split(' and ') for item in mixed_conditions]
  unique_conditions=[]
  for conditions in mixed_conditions:
    for condition in conditions:
      unique_conditions.append(condition)
  return unique_conditions

def filter_unique(single_conditions):
  unique_conditions = set()
  for condition in single_conditions:
    unique_conditions.add(condition)
  return list(unique_conditions)

def clean_data(dataset):
  dataset=[data.lower() for data in dataset]
  translator = str.maketrans('', '', string.punctuation)
  return [data.translate(translator) for data in dataset]

def cleaning_action_stubs(mixed_actions):
  return filter_unique(clean_data(mixed_actions))

def cleaning_condition_stubs(mixed_conditions):
  single_conditions= create_single_conditions(clean_data(mixed_conditions))
  return filter_unique(single_conditions)

def cleaning_stubs(mixed_conditions,mixed_actions):
  return cleaning_condition_stubs(mixed_conditions),cleaning_action_stubs(mixed_actions)

def get_stubs(sentences):
  condition_stub = []
  action_stub = []
  for sentence in sentences:
      words = sentence.split()
      if words[0].lower() == 'if':
          condition_index = sentence.lower().index('if') + 2
          then_index = sentence.lower().index('then')
          condition_stub.append(sentence[condition_index:then_index].strip())
          action_stub.append(sentence[then_index + 4:].strip())
      else:
          if_index = sentence.lower().index('if')
          condition_stub.append(sentence[if_index + 2:].strip())
          action_stub.append(sentence[:if_index].strip())

  conditions,actions =(cleaning_stubs(condition_stub,action_stub))
  return conditions,actions


# **Rule Generation**

In [122]:
def get_conditions_for_a_rule(mixed_condition,condition_stubs):
  conditions_for_a_rule=[]

  conditions=split_mixed_stub(mixed_condition)
  for condition in conditions:
    condition=condition.strip()
    for index, condition_stub in enumerate(condition_stubs):
      if condition in condition_stub:
        conditions_for_a_rule.append(index)
  return conditions_for_a_rule

def get_actions_for_a_rule(mixed_action,action_stubs):
  actions_for_a_rule=[]

  actions=split_mixed_stub(mixed_action)
  for action in actions:
    action=action.strip()
    for index, action_stub in enumerate(action_stubs):
      if action in action_stub:
        actions_for_a_rule.append(index)
  return actions_for_a_rule

def parse_sentence(sentence,if_then):
  condition_index = sentence.index('if')
  if(if_then):
    then_index = sentence.index('then')
    mixed_condition=sentence[condition_index+2:then_index].strip()
    mixed_action=sentence[then_index + 4:].strip()
  else:
    mixed_condition=sentence[condition_index+2:].strip()
    mixed_action=sentence[:condition_index].strip()
  return mixed_condition,mixed_action

def get_condition_and_action_from_sentence(sentence):
  sentence=''.join(clean_data(sentence))
  words = sentence.split()

  if words[0] == 'if':
    mixed_condition,mixed_action=parse_sentence(sentence,True)
  else:
    mixed_condition,mixed_action=parse_sentence(sentence,False)
  return mixed_condition,mixed_action

def split_mixed_stub(mixed_condition):
  return mixed_condition.split('and')

def create_rules(sentences,condition_stubs,action_stubs):

  rules={}
  rule=1
  for sentence in sentences:

    mixed_condition,mixed_action =  get_condition_and_action_from_sentence(sentence)
    conditions_for_a_rule =   get_conditions_for_a_rule(mixed_condition,condition_stubs)
    actions_for_a_rule =   get_actions_for_a_rule(mixed_action,action_stubs)

    rules[rule]=[conditions_for_a_rule,actions_for_a_rule]
    rule+=1
  return rules


# **Table Generation**

In [123]:
def make_column_headers(rules):
  column_headers = []
  for i in rules:
    column_header="R"+str(i)
    column_headers.append(column_header)
  return column_headers

def make_row_headers(condition_stubs,action_stubs):
  row_headers = []

  for index,condition_stub in enumerate(condition_stubs):
    row_header="C"+str(index+1)+":"+str(condition_stub)
    row_headers.append(row_header)

  for index,action_stub in enumerate(action_stubs):
    row_header="A"+str(index+1)+":"+str(action_stub)
    row_headers.append(row_header)

  return row_headers

def make_table(condition_stubs,action_stubs,rules):
  column_headers=make_column_headers(rules)
  row_headers=make_row_headers(condition_stubs,action_stubs)
  table = pd.DataFrame(columns=column_headers, index=row_headers)
  return table

def initialize_table(table):
  for row in table.index:
      if(row.startswith('C')):
        for col in table.columns:
                table.at[row, col] = 'F'
      else:
        for col in table.columns:
          table.at[row, col] = ' '
  return table

def set_condition_stubs_entries(table,rules):
  for i in range(1,len(table.columns)+1):
    condition_set=(rules[i][0])
    for condition in condition_set:
      for row in table.index:
        if(row.startswith('C') and int(row[1])==condition+1):
          col=table.columns[i-1]
          table.at[row,col] ='T'
  return table

def set_action_stubs_entries(table,rules):
  for i in range(1,len(table.columns)+1):
    action_set=(rules[i][1])
    for action in action_set:
      for row in table.index:
        if(row.startswith('A') and int(row[1])==action+1):
          col=table.columns[i-1]
          table.at[row,col] ='x'
  return table

def generate_table(condition_stubs,action_stubs,rules):
  table=initialize_table(make_table(condition_stubs,action_stubs,rules))
  table=set_action_stubs_entries(table,rules)
  table=set_condition_stubs_entries(table,rules)
  return table

def printDetails(condition_stubs,action_stubs,rules,table):
  print("Rules are: ")
  for rule in rules:

    mixed_condition=""
    for condition in rules[rule][0]:
      mixed_condition+=condition_stubs[condition]
      mixed_condition+=','


    mixed_action=""
    for action in rules[rule][1]:
      mixed_action+=action_stubs[action]
      mixed_action+=','
    print(str(rule)+":"+"If "+mixed_condition+"then "+mixed_action)


  print()
  print(table)


# **Test Case generation**
## Task-1
We need to parse the condition and action like this:<br>input: the order is from dgsd
<br>output: order=dgsd

<br>input: the order is from agents
<br>output: order=agents

<br>input: the order is from retailer
<br>output: order=retailer

<br>input: the amount is more than rs 50000
<br>output: amount>=50000


<br>input: the amount is between rs 20000 to rs 50000
<br>output: amount>=20000 &&  amount<=50000


<br>input: the amount is less than rs 20000
<br>output: amount<=20000

<br>input: item is furniture
<br>output: item=furniture

<br>input: the discount is 15
<br>output: discount=15

<br>

## Task-2

From there we need to find out the dependency/toggling of the variables and we will optimize our table

##Task-3

Finally we will generate the test cases after we are finished with optimizing our decision table



In [139]:
def prune_determinant(sentences):

  nlp = spacy.load("en_core_web_sm")
  pruned_sentences=[]
  for sentence in sentences:
    doc = nlp(sentence)
    sentence = " ".join([token.text for token in doc if token.pos_ != "DET"])
    pruned_sentences.append(sentence)
  return pruned_sentences

def describe_parts_of_speech(sentence):

  nlp = spacy.load("en_core_web_sm")
  parts_of_speech={}
  doc = nlp(sentence)
  for token in doc:
    parts_of_speech[token.text]=token.pos_
  return parts_of_speech

def noun_list(sentence):
  nlp = spacy.load("en_core_web_sm")
  nouns=[]
  doc = nlp(sentence)
  for token in doc:
    if(token.pos_=='NOUN'):
      nouns.append(token.text)
  return nouns


def convert_to_expression(sentences):
  expression=''
  return expression

condition_stubs=prune_determinant(condition_stubs)
for condition in condition_stubs:
  print(describe_parts_of_speech(condition))

{'order': 'NOUN', 'is': 'AUX', 'from': 'ADP', 'retailer': 'NOUN'}
{'item': 'NOUN', 'is': 'AUX', 'furniture': 'NOUN'}
{'amount': 'NOUN', 'is': 'AUX', 'less': 'ADJ', 'than': 'ADP', 'rs': 'ADJ', '20000': 'NUM'}
{'order': 'NOUN', 'is': 'AUX', 'from': 'ADP', 'dgsd': 'NOUN'}
{'amount': 'NOUN', 'is': 'AUX', 'between': 'ADP', 'rs': 'VERB', '20000': 'NUM', 'to': 'PART', '50000': 'NUM'}
{'order': 'NOUN', 'is': 'AUX', 'from': 'ADP', 'agents': 'NOUN'}
{'amount': 'NOUN', 'is': 'AUX', 'more': 'ADJ', 'than': 'ADP', 'rs': 'ADJ', '50000': 'NUM'}


# **Driver**

In [135]:
condition_stubs,action_stubs=get_stubs(sentences)
rules=create_rules(sentences,condition_stubs,action_stubs)
table=generate_table(condition_stubs,action_stubs,rules)

printDetails(condition_stubs,action_stubs,rules,table)


Rules are: 
1:If the order is from dgsd,then the discount is 10,
2:If the order is from agents,the amount is more than rs 50000,then the discount is 15,
3:If the order is from retailer,the amount is more than rs 50000,then the discount is 10,
4:If the amount is between rs 20000 to rs 50000,the order is from agents,then the discount is 12,
5:If the amount is between rs 20000 to rs 50000,the order is from retailer,then the discount is 8,
6:If the amount is less than rs 20000,the order is from agents,then the discount is 8,
7:If the order is from retailer,the amount is less than rs 20000,then the discount is 5,
8:If item is furniture,then the discount is 10,

                                              R1 R2 R3 R4 R5 R6 R7 R8
C1:the order is from retailer                  F  F  T  F  T  F  T  F
C2:item is furniture                           F  F  F  F  F  F  F  T
C3:the amount is less than rs 20000            F  F  F  F  F  T  T  F
C4:the order is from dgsd                      T  F  F 

In [126]:
import difflib

sentence1 = "the amount is more than rs 50000"
sentence2 = "the amount is less than rs 20000"

similarity_percentage = difflib.SequenceMatcher(None, sentence1, sentence2).ratio()*100

print(f"Similarity Percentage: {similarity_percentage:.2f}%")

Similarity Percentage: 87.50%


In [ ]:


nlp = spacy.load("en_core_web_sm")

# Your input sentence
input_sentence = "the working hours are 48 or less"

# Define custom rules to convert text to mathematical expressions
conversion_rules = {
    "less than": "<",
    "greater than": ">",
    "or": "or",
    "and": "and",
    "is":"=",
    "are":"=",
}

# Tokenize and process the input sentence
doc = nlp(input_sentence)

# Initialize an empty expression string
math_expression = ""

# Iterate through the tokens and apply custom rules
for token in doc:
    if token.text in conversion_rules:
        math_expression += f" {conversion_rules[token.text]} "
    elif token.is_digit:
        math_expression += token.text
    elif token.is_alpha:
        math_expression += token.text  # Assuming variables as-is
    else:
        math_expression += token.text

print(f"Input: '{input_sentence}'")
print(f"Mathematical Expression: {math_expression}")
